<a href="https://colab.research.google.com/github/ThisIsFarhan/pytorch-codes/blob/main/05_TrainingPipeline_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")

In [3]:
encoder = LabelEncoder()
encoder.fit(df["species"])
df["encoded_species"] = encoder.transform(df["species"])

In [4]:
df = df[df["encoded_species"] != 2]

In [6]:
df["encoded_species"].value_counts()

,count
encoded_species,
0,50
1,50


In [7]:
df

,sepal_length,sepal_width,petal_length,petal_width,species,encoded_species
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,versicolor,1
96,5.7,2.9,4.2,1.3,versicolor,1
97,6.2,2.9,4.3,1.3,versicolor,1
98,5.1,2.5,3.0,1.1,versicolor,1


In [8]:
df.drop(columns=["species"],inplace=True)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_train

array([[-0.70610559, -0.22852704, -0.87502229, -1.04130681],
       [-0.55091755,  1.53360914, -0.87502229, -1.04130681],
       [ 0.3802107 ,  1.53360914, -0.80672786, -0.86739335],
       [ 0.22502266, -1.32986215,  0.69574943,  0.52391428],
       [ 1.93209112, -0.00826001,  1.03722154,  1.04565464],
       [-0.08535342,  1.31334212, -0.94331671, -1.04130681],
       [ 1.15615091, -0.44879406,  0.96892712,  0.87174119],
       [ 0.53539874, -0.8893281 ,  0.83233827,  0.35000083],
       [-0.86129363, -1.55012917,  0.28598289,  0.35000083],
       [ 0.53539874, -1.10959512,  0.76404385,  0.69782774],
       [-0.55091755, -1.32986215,  0.08109963,  0.52391428],
       [-1.32685776,  0.21200701, -1.01161113, -1.04130681],
       [-1.63723384, -0.44879406, -1.01161113, -1.04130681],
       [ 1.31133895, -1.32986215,  1.37869365,  1.2195681 ],
       [-1.01648167,  0.65254105, -0.67013902, -1.04130681],
       [ 0.06983462, -1.32986215,  0.76404385,  0.87174119],
       [-0.70610559,  0.

In [18]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.to_numpy().astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.to_numpy().astype(np.float32))

In [19]:
X_train_tensor.shape

torch.Size([80, 4])

In [20]:
X_test_tensor.shape

torch.Size([20, 4])

In [21]:
import torch.nn as nn

class Model(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

In [22]:
learning_rate = 0.1
epochs = 25
loss_function = nn.BCELoss()

In [24]:
model = Model(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  #forward pass
  y_pred = model(X_train_tensor)

  #loss
  loss = loss_function(y_pred.squeeze(), y_train_tensor)

  # clear gradients
  optimizer.zero_grad()

  #backward pass
  loss.backward()

  #params update
  optimizer.step()

  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')


Epoch: 1, Loss: 0.7308187484741211
Epoch: 2, Loss: 0.6597815155982971
Epoch: 3, Loss: 0.5987181663513184
Epoch: 4, Loss: 0.5461111068725586
Epoch: 5, Loss: 0.5006340742111206
Epoch: 6, Loss: 0.4611549377441406
Epoch: 7, Loss: 0.4267217516899109
Epoch: 8, Loss: 0.396541029214859
Epoch: 9, Loss: 0.3699546754360199
Epoch: 10, Loss: 0.34641799330711365
Epoch: 11, Loss: 0.32547980546951294
Epoch: 12, Loss: 0.30676573514938354
Epoch: 13, Loss: 0.28996431827545166
Epoch: 14, Loss: 0.2748156487941742
Epoch: 15, Loss: 0.26110154390335083
Epoch: 16, Loss: 0.2486386001110077
Epoch: 17, Loss: 0.23727157711982727
Epoch: 18, Loss: 0.2268686294555664
Epoch: 19, Loss: 0.21731731295585632
Epoch: 20, Loss: 0.2085212916135788
Epoch: 21, Loss: 0.20039761066436768
Epoch: 22, Loss: 0.19287462532520294
Epoch: 23, Loss: 0.1858901083469391
Epoch: 24, Loss: 0.17938995361328125
Epoch: 25, Loss: 0.17332682013511658


In [25]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5
